# 【AAPlot for Animal Behavior (Analyze)】
## Extract speed and trajectory of **Spike2** exported .txt file

This notebook analyzes animal movement data before and after specific events, including:
- Data cleaning (removal of artifacts)
- Time rescaling around events
- Speed analysis in different time windows
- Distance calculations
- Statistical analysis and data export


Run under `PLOT` environment
    
The `PLOT` enviorment：
- Python3.12.7
- pandas
- numpy
- matplotlib
- seaborn
- ipykernel

*Warning*

*! Make sure you have installed `Anaconda`，and added to PATH（refering to internet）*

*! Make sure you have already confiured the `PLOT` environment, if not, run this command: `conda env create -n PLOT python=3.12.7 pandas numpy matplotlib seaborn ipykernel` (If you are using ARM64 CPU, use Python3.13.3 intead，add `conda-forge` at the end of command)*

*Apply `PLOT` in VScode ：Select in the Kernel*

*Apply `PLOT` in terminal：`conda activate PLOT`*

## 1. Import Libraries and Load Data

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import stats
import glob
from pathlib import Path

# Folder selection - replace with your folder path
folder_path = r'D:\Temp\DrugIntake behavior\THC\BatchAnalysis\0.1'

# Find all CSV and TXT files in the folder
data_files = []
for ext in ['.txt', '.csv']:
    data_files.extend(glob.glob(os.path.join(folder_path, f'*{ext}')))

if not data_files:
    print(f"No .txt or .csv files found in {folder_path}")
    raise ValueError("No data files found")

print(f"Found {len(data_files)} data files:")
for file in data_files:
    print(f"- {os.path.basename(file)}")

# Function to read and preprocess a single file
def read_data_file(filepath):
    # First, look at the file structure
    with open(filepath, 'r') as file:
        first_few_lines = [next(file) for _ in range(5)]
    print(f"\nReading file: {os.path.basename(filepath)}")
    print("First few lines:")
    for line in first_few_lines:
        print(line.strip())
    
    # Try reading with different separators
    for separator in ['\t', ',', '\s+']:
        try:
            df = pd.read_csv(filepath, sep=separator, header=None, skiprows=1)
            if len(df.columns) >= 2:  # We need at least time and speed columns
                print(f"Successfully read file with '{separator}' separator")
                break
        except:
            continue
    else:
        raise ValueError(f"Could not read file {filepath} with any known separator")

    print("Initial data shape:", df.shape)
    
    # Convert time column to numeric
    df.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')
    
    # Rename columns by position
    if len(df.columns) >= 3:
        df = df.iloc[:, :3]  # Take only first three columns
        df.columns = ['Time', 'Speed', 'Marker']
    elif len(df.columns) == 2:
        df = df.iloc[:, :2]  # Take only first two columns
        df.columns = ['Time', 'Speed']
        # Add marker column with default event at middle point
        df['Marker'] = 0
        middle_idx = len(df) // 2
        df.loc[middle_idx, 'Marker'] = 1
        print("No marker column found. Added marker at middle point.")
    else:
        raise ValueError("File must have at least 2 columns (Time and Speed)")
    
    return df

# Process the first file as an example
if data_files:
    example_df = read_data_file(data_files[0])
    print("\nExample file structure:")
    print(example_df.info())
    print("\nSample of processed data:")
    print(example_df.head())

<>:36: SyntaxWarning: invalid escape sequence '\s'
<>:36: SyntaxWarning: invalid escape sequence '\s'
C:\Users\RuyiCai\AppData\Local\Temp\ipykernel_39592\1143523068.py:36: SyntaxWarning: invalid escape sequence '\s'
  for separator in ['\t', ',', '\s+']:


Found 7 data files:
- C16_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
- C17_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
- C19_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
- C93 0.1mpk.txt
- C94 0.1mpk.txt
- C95 0.1mpk.txt
- C96 0.1mpk.txt

Reading file: C16_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
First few lines:
"Time","2 Speed(cm/s)","1 Channel 1"
0.0,12.39,0
0.1,10.70,0
0.2,9.42,0
0.3,8.57,0
Successfully read file with ',' separator
Initial data shape: (417008, 3)

Example file structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417008 entries, 0 to 417007
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    417008 non-null  float64
 1   Speed   417008 non-null  float64
 2   Marker  417008 non-null  int64  
dtypes: float64(2), int64(1)
memory usage: 9.5 MB
None

Sample of processed data:
   Time  Speed  Marker
0   0.0  12.39       0
1   0.1  10.70       0
2   0.2   9.42       0
3   0.3   8.57       0
4  

## 2. Data Cleaning and Preprocessing

In [60]:
# Data cleaning
try:
    df
except NameError:
    # If df is not defined, use example_df from the previous cell
    df = example_df
print("Original data shape:", df.shape)

# Convert speed column to numeric, handling any non-numeric values
df_cleaned = df.copy()
# Speed is always in column 1 (second column)
df_cleaned.iloc[:, 1] = pd.to_numeric(df_cleaned.iloc[:, 1], errors='coerce')
print("\nInitial speed statistics:")
print(df_cleaned.iloc[:, 1].describe())

# Remove speed artifacts (>100 cm/s)
artifacts_mask = df_cleaned.iloc[:, 1] > 100
n_artifacts = artifacts_mask.sum()
df_cleaned.iloc[artifacts_mask, 1] = np.nan  # Replace artifacts with NaN
print(f"\nNumber of artifacts removed (speed > 100 cm/s): {n_artifacts}")
print("\nSpeed statistics after removing artifacts:")
print(df_cleaned.iloc[:, 1].describe())

# Interpolate NaN values
df_cleaned.iloc[:, 1] = df_cleaned.iloc[:, 1].interpolate(method='linear')
print("\nSpeed statistics after interpolation:")
print(df_cleaned.iloc[:, 1].describe())

# Verify event markers (using position-based indexing for consistency)
event_times = df_cleaned[df_cleaned.iloc[:, 2] == 1].iloc[:, 0]  # Column 2 is Marker, Column 0 is Time
print(f"\nNumber of events found: {len(event_times)}")
print("Event time points:", event_times.values)

# Calculate percentiles for speed distribution
percentiles = [5, 25, 50, 75, 95]
speed_percentiles = np.percentile(df_cleaned.iloc[:, 1].dropna(), percentiles)
print("\nSpeed distribution percentiles:")
for p, v in zip(percentiles, speed_percentiles):
    print(f"{p}th percentile: {v:.2f} cm/s")

Original data shape: (115590, 3)

Initial speed statistics:
count    115590.000000
mean          7.437594
std          12.189164
min         -36.187000
25%           2.186000
50%           4.431000
75%           8.372000
max         311.414000
Name: Speed, dtype: float64

Number of artifacts removed (speed > 100 cm/s): 370

Speed statistics after removing artifacts:
count    115220.000000
mean          6.973161
std           8.729149
min         -36.187000
25%           2.186000
50%           4.419000
75%           8.312000
max          99.545000
Name: Speed, dtype: float64

Speed statistics after interpolation:
count    115590.000000
mean          7.215625
std           9.728024
min         -36.187000
25%           2.186000
50%           4.431000
75%           8.372000
max          99.545000
Name: Speed, dtype: float64

Number of events found: 1
Event time points: [3149.2]

Speed distribution percentiles:
5th percentile: 0.90 cm/s
25th percentile: 2.19 cm/s
50th percentile: 4.43 cm/s


In [61]:
def analyze_locomotion_data(df, filename):
    """
    Analyze locomotion data for a single file
    
    Parameters:
    df : pandas DataFrame
        Raw data with Time, Speed, and Marker columns
    filename : str
        Original filename for reporting
        
    Returns:
    tuple : (df_cleaned, speed_results, distance_results, data_completeness)
    """
    print(f"\nAnalyzing file: {os.path.basename(filename)}")
    
    # Data cleaning
    print("Original data shape:", df.shape)
    
    df_cleaned = df.copy()
    df_cleaned.iloc[:, 1] = pd.to_numeric(df_cleaned.iloc[:, 1], errors='coerce')
    
    # Remove speed artifacts (>100 cm/s)
    artifacts_mask = df_cleaned.iloc[:, 1] > 100
    n_artifacts = artifacts_mask.sum()
    df_cleaned.iloc[artifacts_mask, 1] = np.nan
    print(f"Number of artifacts removed (speed > 100 cm/s): {n_artifacts}")
    
    # Interpolate NaN values
    df_cleaned.iloc[:, 1] = df_cleaned.iloc[:, 1].interpolate(method='linear')
    
    # Find the event time
    try:
        event_markers = df_cleaned[df_cleaned.iloc[:, 2] == 1]
        if len(event_markers) > 0:
            event_time = event_markers.iloc[0, 0]
            print(f"Found event marker at time: {event_time:.2f} seconds")
        else:
            time_min = df_cleaned.iloc[:, 0].min()
            time_max = df_cleaned.iloc[:, 0].max()
            event_time = (time_max + time_min) / 2
            print(f"No event markers found. Using middle point as event time: {event_time:.2f} seconds")
    except Exception as e:
        print(f"Error finding event time: {str(e)}")
        time_min = df_cleaned.iloc[:, 0].min()
        time_max = df_cleaned.iloc[:, 0].max()
        event_time = (time_max + time_min) / 2
        print(f"Using middle point as event time: {event_time:.2f} seconds")

    # Rescale time relative to event (in hours)
    df_cleaned['Time_hours'] = (df_cleaned.iloc[:, 0] - event_time) / 3600

    # Define time windows (corrected to cumulative post-event windows)
    time_windows = {
        'pre_1h': (-1.1, -0.1),    # 1 hour before event
        'post_1h': (0.1, 1.1),    # 0-1 hour after event
        'post_2h': (0.1, 2.1),    # 0-2 hours after event
        'post_3h': (0.1, 3.1)     # 0-3 hours after event
    }
    
    # Calculate average speeds for each time window
    speed_results = {}
    data_completeness = {}
    
    for window_name, (start, end) in time_windows.items():
        mask = (df_cleaned['Time_hours'] >= start) & (df_cleaned['Time_hours'] < end)
        window_data = df_cleaned[mask]
        
        if len(window_data) > 0:
            speed_results[window_name] = {
                'mean_speed': window_data['Speed'].mean(),
                'std_speed': window_data['Speed'].std(),
                'n_points': len(window_data)
            }
            time_coverage = (window_data['Time_hours'].max() - window_data['Time_hours'].min()) * 60
            data_completeness[window_name] = min(100, (time_coverage / 60) * 100)
        else:
            speed_results[window_name] = {
                'mean_speed': np.nan,
                'std_speed': np.nan,
                'n_points': 0
            }
            data_completeness[window_name] = 0
    
    # Calculate distances
    distance_results = {}
    for window_name, (start, end) in time_windows.items():
        mask = (df_cleaned['Time_hours'] >= start) & (df_cleaned['Time_hours'] < end)
        window_data = df_cleaned[mask]
        
        if len(window_data) > 0:
            speeds_cmh = window_data['Speed'] * 3600
            time_diff_h = np.diff(window_data['Time_hours'])
            speeds_for_calc = speeds_cmh[:-1]
            
            distance = np.sum(speeds_for_calc * time_diff_h)
            
            distance_results[window_name] = {
                'distance_cm': distance,
                'distance_m': distance / 100,
                'completeness': data_completeness[window_name]
            }
        else:
            distance_results[window_name] = {
                'distance_cm': np.nan,
                'distance_m': np.nan,
                'completeness': 0
            }
    
    return df_cleaned, speed_results, distance_results, data_completeness

## 3. Time Rescaling

In [62]:
# Find the event time
try:
    # Using position-based indexing for consistency
    event_markers = df_cleaned[df_cleaned.iloc[:, 2] == 1]
    if len(event_markers) > 0:
        event_time = event_markers.iloc[0, 0]  # Get time from first event marker
        print(f"Found event marker at time: {event_time:.2f} seconds")
    else:
        # If no event markers found, use middle of the time range
        time_min = df_cleaned.iloc[:, 0].min()
        time_max = df_cleaned.iloc[:, 0].max()
        event_time = (time_max + time_min) / 2
        print(f"No event markers found. Using middle point as event time: {event_time:.2f} seconds")
except Exception as e:
    print(f"Error finding event time: {str(e)}")
    # Default to middle of time range
    time_min = df_cleaned.iloc[:, 0].min()
    time_max = df_cleaned.iloc[:, 0].max()
    event_time = (time_max + time_min) / 2
    print(f"Using middle point as event time: {event_time:.2f} seconds")

# Rescale time relative to event (in hours)
df_cleaned['Time_hours'] = (df_cleaned.iloc[:, 0] - event_time) / 3600  # Convert to hours

# Create time windows for analysis
time_windows = {
    'pre_1h': (-1, 0),
    'post_1h': (0, 1),
    'post_2h': (1, 2),
    'post_3h': (2, 3)
}

# Print time range information
print("\nTime range in dataset:")
print(f"Start: {df_cleaned['Time_hours'].min():.2f} hours")
print(f"End: {df_cleaned['Time_hours'].max():.2f} hours")
print(f"Total duration: {df_cleaned['Time_hours'].max() - df_cleaned['Time_hours'].min():.2f} hours")

# Print data points in each window
print("\nData coverage in time windows:")
for window_name, (start, end) in time_windows.items():
    window_data = df_cleaned[(df_cleaned['Time_hours'] >= start) & (df_cleaned['Time_hours'] < end)]
    print(f"{window_name}: {len(window_data)} data points")

Found event marker at time: 3149.20 seconds

Time range in dataset:
Start: -0.87 hours
End: 2.34 hours
Total duration: 3.21 hours

Data coverage in time windows:
pre_1h: 31492 data points
post_1h: 36000 data points
post_2h: 36000 data points
post_3h: 12098 data points


## 4. Speed Analysis

In [63]:
# Calculate average speeds for each time window
speed_results = {}
data_completeness = {}

for window_name, (start, end) in time_windows.items():
    mask = (df_cleaned['Time_hours'] >= start) & (df_cleaned['Time_hours'] < end)
    window_data = df_cleaned[mask]
    
    if len(window_data) > 0:
        speed_results[window_name] = {
            'mean_speed': window_data['Speed'].mean(),
            'std_speed': window_data['Speed'].std(),
            'n_points': len(window_data)
        }
        # Calculate data completeness (percentage of the full hour covered)
        time_coverage = (window_data['Time_hours'].max() - window_data['Time_hours'].min()) * 60  # in minutes
        data_completeness[window_name] = min(100, (time_coverage / 60) * 100)  # as percentage of an hour
    else:
        speed_results[window_name] = {
            'mean_speed': np.nan,
            'std_speed': np.nan,
            'n_points': 0
        }
        data_completeness[window_name] = 0

# Print results
print("Speed Analysis Results:")
for window, results in speed_results.items():
    print(f"\n{window}:")
    print(f"Mean Speed: {results['mean_speed']:.2f} cm/s")
    print(f"Std Dev: {results['std_speed']:.2f} cm/s")
    print(f"Data Points: {results['n_points']}")
    print(f"Data Completeness: {data_completeness[window]:.1f}%")

Speed Analysis Results:

pre_1h:
Mean Speed: 11.55 cm/s
Std Dev: 13.05 cm/s
Data Points: 31492
Data Completeness: 87.5%

post_1h:
Mean Speed: 6.19 cm/s
Std Dev: 6.56 cm/s
Data Points: 36000
Data Completeness: 100.0%

post_2h:
Mean Speed: 4.51 cm/s
Std Dev: 3.75 cm/s
Data Points: 36000
Data Completeness: 100.0%

post_3h:
Mean Speed: 7.05 cm/s
Std Dev: 14.74 cm/s
Data Points: 12098
Data Completeness: 33.6%


## 5. Distance Calculation

In [64]:
# Calculate distances for each time window
distance_results = {}

for window_name, (start, end) in time_windows.items():
    mask = (df_cleaned['Time_hours'] >= start) & (df_cleaned['Time_hours'] < end)
    window_data = df_cleaned[mask]
    
    if len(window_data) > 0:
        # Calculate distance by integrating speed over time
        # Convert speed from cm/s to cm/h and time differences to hours
        speeds_cmh = window_data['Speed'] * 3600  # convert to cm/h
        time_diff_h = np.diff(window_data['Time_hours'])
        speeds_for_calc = speeds_cmh[:-1]  # use speeds except last point
        
        # Calculate distance
        distance = np.sum(speeds_for_calc * time_diff_h)
        
        distance_results[window_name] = {
            'distance_cm': distance,
            'distance_m': distance / 100,  # convert to meters
            'completeness': data_completeness[window]
        }
    else:
        distance_results[window_name] = {
            'distance_cm': np.nan,
            'distance_m': np.nan,
            'completeness': 0
        }

# Print results
print("Distance Analysis Results:")
for window, results in distance_results.items():
    print(f"\n{window}:")
    print(f"Distance: {results['distance_m']:.2f} meters")
    print(f"Data Completeness: {results['completeness']:.1f}%")

Distance Analysis Results:

pre_1h:
Distance: 363.53 meters
Data Completeness: 33.6%

post_1h:
Distance: 222.73 meters
Data Completeness: 33.6%

post_2h:
Distance: 162.44 meters
Data Completeness: 33.6%

post_3h:
Distance: 85.19 meters
Data Completeness: 33.6%


## 6. Data Export

In [65]:
# Create output directory for batch analysis
output_dir = os.path.join(folder_path, 'analysis_results')
os.makedirs(output_dir, exist_ok=True)

# Process all files
all_results = []

for file_path in data_files:
    try:
        print(f"\nProcessing file: {os.path.basename(file_path)}")
        
        # Read and analyze the file
        df = read_data_file(file_path)
        df_cleaned, speed_results, distance_results, data_completeness = analyze_locomotion_data(df, file_path)
        
        # Clean up filename by removing "_LocationOutput_TimeOverSpeed.txt"
        base_name = os.path.splitext(os.path.basename(file_path))[0]
        clean_name = base_name.replace("_LocationOutput_TimeOverSpeed", "")
        
        # Create summary for this file as a single row
        file_result = {
            'File': clean_name,
            'Total_Duration_hours': df_cleaned['Time_hours'].max() - df_cleaned['Time_hours'].min(),
            'Pre_event_Duration_hours': abs(df_cleaned['Time_hours'].min()),
            'Post_event_Duration_hours': df_cleaned['Time_hours'].max(),
            'Speed_Pre_1h': speed_results['pre_1h']['mean_speed'],
            'Speed_Post_1h': speed_results['post_1h']['mean_speed'],
            'Speed_Post_2h': speed_results['post_2h']['mean_speed'],
            'Speed_Post_3h': speed_results['post_3h']['mean_speed'],
            'Distance_Pre_1h': distance_results['pre_1h']['distance_m'],
            'Distance_Post_1h': distance_results['post_1h']['distance_m'],
            'Distance_Post_2h': distance_results['post_2h']['distance_m'],
            'Distance_Post_3h': distance_results['post_3h']['distance_m']
        }
        
        # Save cleaned data (use original base_name for data files)
        data_csv = os.path.join(output_dir, f"{base_name}_data.csv")
        df_cleaned.to_csv(data_csv, index=False)
        
        # Add to all results
        all_results.append(file_result)
        
        print(f"Results saved for {clean_name}")
        
    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {str(e)}")
        continue

# Create and save combined summary
if all_results:
    # Convert results to DataFrame (each file is a row)
    combined_summary = pd.DataFrame(all_results)
    
    # Save combined summary
    combined_csv = os.path.join(output_dir, "combined_summary.csv")
    combined_summary.to_csv(combined_csv, index=False)
    
    print(f"\nBatch analysis complete!")
    print(f"Results saved in: {output_dir}")
    print(f"Files processed: {len(all_results)} out of {len(data_files)}")
    
    # Display combined summary
    print("\nCombined Summary (first few rows):")
    print(combined_summary.head().to_string())
else:
    print("\nNo files were successfully processed")


Processing file: C16_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt

Reading file: C16_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
First few lines:
"Time","2 Speed(cm/s)","1 Channel 1"
0.0,12.39,0
0.1,10.70,0
0.2,9.42,0
0.3,8.57,0
Successfully read file with ',' separator
Initial data shape: (417008, 3)

Analyzing file: C16_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
Original data shape: (417008, 3)
Number of artifacts removed (speed > 100 cm/s): 186
Found event marker at time: 7109.10 seconds
Results saved for C16_THC_0_1mpk

Processing file: C17_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt

Reading file: C17_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
First few lines:
"Time","2 Speed(cm/s)","1 Channel 1"
0.0,4.263,0
0.1,6.686,0
0.2,8.617,0
0.3,10.057,0
Successfully read file with ',' separator
Initial data shape: (401418, 3)

Analyzing file: C17_THC_0_1mpk_LocationOutput_TimeOverSpeed.txt
Original data shape: (401418, 3)
Number of artifacts removed (speed > 100 cm/s): 100
Found event ma